In [1]:
import cv2

In [3]:
gray_image=cv2.imread('What-Colors-Make-Gray.jpg')

colorized_image=cv2.applyColorMap(gray_image, cv2.COLORMAP_JET)

cv2.imwrite('output.jpg', colorized_image)

True

In [1]:
import cv2
import numpy as np

config_path = "yolov3.cfg"
weights_path = "yolov3.weights"
class_path = "coco.names"

with open(class_path, 'r') as f:
    classes = f.read().strip().split('\n')

net = cv2.dnn.readNet(config_path, weights_path)

image = cv2.imread("paris.jpg")
height, width = image.shape[:2]

blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
outs = net.forward(net.getUnconnectedOutLayersNames())

human_mask = np.zeros((height, width), dtype=np.uint8)

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5 and classes[class_id] == 'person':
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            human_mask[y:y + h, x:x + w] = 255

blurred_mask = cv2.GaussianBlur(human_mask, (51, 51), 0)
colorized_mask = cv2.applyColorMap(blurred_mask, cv2.COLORMAP_JET)

masked_background = cv2.bitwise_and(image, image, mask=~blurred_mask)
result = cv2.addWeighted(masked_background, 1, colorized_mask, 1, 0)

cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: 'coco.names'